## Offensive and defensive ELO ratings to predict number of goals

We keep track of two ratings for all teams offensive rating ($R_O$) and defensive rating ($R_D$).

We can then predict the number of goals a team will score by taking the difference of their offensive rating and the opponent's defensive rating.

The number of goals scored against them can be calculated by considering it from the opponent's perspective.

$E[\text{team}] = R_O[\text{team}] - R_D[\text{opponent}]$

We can update a team's offensive rating by adding the difference between the actual number of goals and the expected goals multiplied by the learning rate.
We can update a team's defensive rating by adding the difference between the expected goals scored against them and the actual number of goals scored against them multiplied by the learning rate.

$R_O[\text{team}] = R_O[\text{team}] + k(G[\text{team}] - E[\text{team}])$

$R_D[\text{team}] = R_D[\text{team}] + k(E[\text{opponent}] - G[\text{opponent}])$

We start every team with a rating of 0. The order of the training data makes a difference to the model and so the training data should be in chronological order in order to account for teams changing over team.


In [24]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm, tqdm_notebook
import sklearn.model_selection
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
match_data = pd.read_csv("data/football-data.co.uk/updated-epl-training.csv")

In [3]:
def encode_result(result):
    if result == 'H':
        return 1
    elif result == 'A':
        return 0
    return 0.5

In [4]:
def decode_result(result):
    if result == 1:
        return 'H'
    elif result == 0:
        return 'A'
    return 'D'

In [28]:
class GoalElo:
    def __init__(self, initial_rating=0, learning_rate=0.05, draw_size=0.5):
        self.offensive_ratings = defaultdict(lambda: initial_rating)
        self.defensive_ratings = defaultdict(lambda: initial_rating)
        self.match_count = defaultdict(lambda: 0)
        self.learning_rate = learning_rate
        self.draw_size = draw_size

    def predict(self, team, opponent):
        ''' Predicts the number of goals team will score against opponent. '''
        return self.offensive_ratings[team] - self.defensive_ratings[opponent]

    def predict_result(self, team, opponent):
        ''' Predicts the result of a match. 1 if team wins, 0 if opponent wins and 0.5 if it is a draw. '''
        goals_scored = self.predict(team, opponent)
        goals_conceded = self.predict(opponent, team)
        return self.classify_result(goals_scored, goals_conceded)

    def classify_result(self, goals_scored, goals_conceded):
        goal_difference = goals_scored - goals_conceded
        # result = round(1 / (1 + 10**(-goal_difference)))
        result = 1 if goal_difference > 0 else 0
        if abs(goal_difference) < self.draw_size:
            result = 0.5
        return result


    def predict_data(self, df):
        out = df.copy()
        for i, row in out.iterrows():
            out.at[i, 'EHG'] = self.predict(row['HomeTeam'], row['AwayTeam'])
            out.at[i, 'EAG'] = self.predict(row['AwayTeam'], row['HomeTeam'])
            out.at[i, 'ER'] = decode_result(self.predict_result(row['HomeTeam'], row['AwayTeam']))
        return out

    def update_match(self, home, away, home_actual_goals, away_actual_goals):
        ''' Updates the offensive and defensive ratings of both teams in a match. '''
        home_expected_goals = self.predict(home, away)
        away_expected_goals = self.predict(away, home)
        self.offensive_ratings[home] += self.learning_rate * (home_actual_goals - home_expected_goals)
        self.offensive_ratings[away] += self.learning_rate * (away_actual_goals - away_expected_goals)
        self.defensive_ratings[home] += self.learning_rate * (away_expected_goals - away_actual_goals)
        self.defensive_ratings[away] += self.learning_rate * (home_expected_goals - home_actual_goals)
        self.match_count[home] += 1
        self.match_count[away] += 1

    def ratings_dataframe(self):
        ''' Creates an easy to read dataframe of the ratings '''
        df = pd.DataFrame(self.offensive_ratings.items(), columns=['Team', 'Offensive Rating'])
        df['Defensive Rating'] = df['Team'].map(self.defensive_ratings)
        df['Matches'] = df['Team'].map(self.match_count)
        df = df.sort_values('Offensive Rating', ascending=False)
        return df

    def fit(self, df):
        ''' Takes a data frame of matches with columns HomeTeam, AwayTeam, FTHG, FTAG and updates teams ratings using the data in order. '''
        for i, row in df.iterrows():
            self.update_match(row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'])

    def test(self, df):
        ''' Takes a data frame of matches with columns HomeTeam, AwayTeam, FTHG, FTAG and uses the ratings to predict the number of goals scored by each side. It measures the average mean square error and average mean absolute error per match. '''
        mse = 0
        mae = 0
        count = 0
        for i, row in df.iterrows():
            error_home = self.predict(row['HomeTeam'], row['AwayTeam']) - row['FTHG']
            error_away = self.predict(row['AwayTeam'], row['HomeTeam']) - row['FTAG']
            mse += error_home ** 2 + error_away ** 2
            mae += abs(error_home) + abs(error_away)
            count += 1
        return mse / count, mae / count

    def test_result(self, df):
        ''' Takes a data frame of matches with columns HomeTeam, AwayTeam, FTR and predicts the outcome using the ratings. It measures the number of correct predictions as a percentage of the size of the data. '''
        correct = 0
        count = 0
        for i, row in df.iterrows():
            result = self.predict_result(row['HomeTeam'], row['AwayTeam'])
            if decode_result(result) == row['FTR']:
                correct += 1
            count += 1
        return correct / count



In [6]:
training, test = sklearn.model_selection.train_test_split(match_data, test_size=0.05, shuffle=False)

In [29]:
goal_elo = GoalElo()
goal_elo.fit(training)

data = goal_elo.predict_data(training)

X = np.array([data['EHG'].to_numpy(), data['EAG'].to_numpy()]).T

y = data['FTR'].to_numpy()

model = SVC(gamma='auto')
model.fit(X, y)

data = goal_elo.predict_data(test)
X_test = np.array([data['EHG'].to_numpy(), data['EAG'].to_numpy()]).T
y_test = data['FTR'].to_numpy()
y_pred = model.predict(X_test)
y_pred2 = data['ER'].to_numpy()

print("DS: ", accuracy_score(y_test, y_pred2))
print("SVC: ", accuracy_score(y_test, y_pred))



DS:  0.4641350210970464
SVC:  0.48945147679324896


In [63]:
learning_rates = np.logspace(-4, -1, num=50)
mse_list = []
mae_list = []
accuracy = []

for learning_rate in tqdm(learning_rates):
    goal_elo = GoalElo(learning_rate=learning_rate)
    goal_elo.fit(training)
    mse, mae = goal_elo.test(test)
    mse_list.append(mse)
    mae_list.append(mae)
    accuracy.append(goal_elo.test_result(test))
print("Accuracy:") 
pd.DataFrame(zip(learning_rates, mse_list, mae_list, accuracy), columns=['Learning Rate', 'Mean Square Error / Match', 'Mean Absolute Error / Match', 'Accuracy'])

100%|██████████| 50/50 [00:17<00:00,  2.93it/s]Accuracy:



,Learning Rate,Mean Square Error / Match,Mean Absolute Error / Match,Accuracy
0,0.000100,6.919693,2.687661,0.240506
1,0.000115,6.854136,2.676597,0.240506
2,0.000133,6.780298,2.664000,0.240506
3,0.000153,6.697405,2.649681,0.240506
4,0.000176,6.604700,2.633436,0.240506
5,0.000202,6.501473,2.615047,0.240506
6,0.000233,6.387103,2.594285,0.240506
7,0.000268,6.261111,2.570912,0.240506
8,0.000309,6.123229,2.544686,0.240506
9,0.000356,5.973465,2.515374,0.240506


In [65]:
draw_sizes = np.logspace(-4, 1, num=50)
mse_list = []
mae_list = []
accuracy = []

for draw_size in tqdm(draw_sizes):
    goal_elo = GoalElo(draw_size=draw_size)
    goal_elo.fit(training)
    mse, mae = goal_elo.test(test)
    mse_list.append(mse)
    mae_list.append(mae)
    accuracy.append(goal_elo.test_result(test))
print("Accuracy:") 
pd.DataFrame(zip(draw_sizes, mse_list, mae_list, accuracy), columns=['Draw Size', 'Mean Square Error / Match', 'Mean Absolute Error / Match', 'Accuracy'])

100%|██████████| 50/50 [00:16<00:00,  3.03it/s]Accuracy:



,Draw Size,Mean Square Error / Match,Mean Absolute Error / Match,Accuracy
0,0.000100,3.565736,2.034976,0.510549
1,0.000126,3.565736,2.034976,0.510549
2,0.000160,3.565736,2.034976,0.510549
3,0.000202,3.565736,2.034976,0.510549
4,0.000256,3.565736,2.034976,0.510549
5,0.000324,3.565736,2.034976,0.510549
6,0.000409,3.565736,2.034976,0.510549
7,0.000518,3.565736,2.034976,0.510549
8,0.000655,3.565736,2.034976,0.510549
9,0.000829,3.565736,2.034976,0.510549


In [9]:
goal_elo = GoalElo(learning_rate=0.1)
goal_elo.train(training)
# print(goal_elo.ratings_dataframe())
print(goal_elo.predict('Man City', 'Chelsea'))
print(goal_elo.predict('Chelsea', 'Man City'))

2.479256968864154
1.1235611586654741
